## Environment
- Host virtualenv: conda create -n monai python=3.7 pytorch torchvision jupyterlab cudatoolkit=11.1 monai monai-deploy-app-sdk -c pytorch -c conda-forge
- Additional requirements: nibabel, pydicom, SimpleITK, typeguard, wget, gdown

In [ ]:
!pip install monai-deploy-app-sdk==0.4 wget monai==0.8.1 pydicom

# Part 1: MONAIDeploy APP Tutorial

### - Deploy APP Structure
<img src="https://github.com/WarrenTseng/monai_endoscopy_tutorial/blob/main/MONAIDeploy/imgs/dep_app_structure.jpg?raw=1" alt="drawing" width="200"/>

### - APP Pipeline
<img src="https://github.com/WarrenTseng/monai_endoscopy_tutorial/blob/main/MONAIDeploy/imgs/app_pipeline.jpg?raw=1" alt="drawing" width="600"/>

## 1. Download the Sample Data

In [ ]:
!mkdir tmp

In [ ]:
from skimage import io
import wget

test_input_path = "./tmp/normal-brain-mri-4.png"
wget.download("https://user-images.githubusercontent.com/1928522/133383228-2357d62d-316c-46ad-af8a-359b56f25c87.png", test_input_path)

print(f"Test input file path: {test_input_path}")

test_image = io.imread(test_input_path)
io.imshow(test_image)

In [ ]:
# Copy a test input file to 'input' folder
!mkdir -p input && rm -rf input/*
!cp {test_input_path} input/

## 2. Creating Operators

In [ ]:
!mkdir simple_imaging_app

### SobelOperator

In [ ]:
%%writefile simple_imaging_app/sobel_operator.py
import monai.deploy.core as md
from monai.deploy.core import (
    Application,
    DataPath,
    ExecutionContext,
    Image,
    InputContext,
    IOType,
    Operator,
    OutputContext,
)

@md.input("image", DataPath, IOType.DISK)
@md.output("image", Image, IOType.IN_MEMORY)
class SobelOperator(Operator):
    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        from skimage import filters, io

        input_path = op_input.get().path
        if input_path.is_dir():
            input_path = next(input_path.glob("*.*"))  # take the first file

        data_in = io.imread(input_path)[:, :, :3]  # discard alpha channel if exists
        data_out = filters.sobel(data_in)

        op_output.set(Image(data_out))

### MedianOperator

In [ ]:
%%writefile simple_imaging_app/median_operator.py
import monai.deploy.core as md
from monai.deploy.core import (
    Application,
    DataPath,
    ExecutionContext,
    Image,
    InputContext,
    IOType,
    Operator,
    OutputContext,
)

@md.input("image", Image, IOType.IN_MEMORY)
@md.output("image", Image, IOType.IN_MEMORY)
class MedianOperator(Operator):
    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        from skimage.filters import median

        data_in = op_input.get().asnumpy()
        data_out = median(data_in)
        op_output.set(Image(data_out))

### GaussianOperator

In [ ]:
%%writefile simple_imaging_app/gaussian_operator.py
import monai.deploy.core as md
from monai.deploy.core import (
    Application,
    DataPath,
    ExecutionContext,
    Image,
    InputContext,
    IOType,
    Operator,
    OutputContext,
)

@md.input("image", Image, IOType.IN_MEMORY)
@md.output("image", DataPath, IOType.DISK)
class GaussianOperator(Operator):
    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        from skimage.filters import gaussian
        from skimage.io import imsave

        data_in = op_input.get().asnumpy()
        data_out = gaussian(data_in, sigma=0.2)

        output_folder = op_output.get().path
        output_path = output_folder / "final_output.png"
        imsave(output_path, data_out)

## 3. Application Class

In [ ]:
%%writefile simple_imaging_app/app.py
import monai.deploy.core as md
from gaussian_operator import GaussianOperator
from median_operator import MedianOperator
from sobel_operator import SobelOperator

from monai.deploy.core import Application


@md.resource(cpu=1)
@md.env(pip_packages=["scikit-image >= 0.17.2", "monai == 0.7.0", "monai-deploy-app-sdk == 0.2.0"])
class App(Application):
    # App's name. <class name>('App') if not specified.
    name = "simple_imaging_app"
    # App's description. <class docstring> if not specified.
    description = "This is a very simple application."
    # App's version. <git version tag> or '0.0.0' if not specified.
    version = "0.1.0"
    def compose(self):
        """This application has three operators.

        Each operator has a single input and a single output port.
        Each operator performs some kind of image processing function.
        """
        sobel_op = SobelOperator()
        median_op = MedianOperator()
        gaussian_op = GaussianOperator()

        self.add_flow(sobel_op, median_op)
        self.add_flow(median_op, gaussian_op)

# Run the application when this file is executed.
if __name__ == "__main__":
    App(do_run=True)

In [ ]:
%%writefile simple_imaging_app/__main__.py
from app import App

if __name__ == "__main__":
    App(do_run=True)

## 4. APP Testing

In [ ]:
!monai-deploy exec simple_imaging_app -i {test_input_path} -o output
# !python simple_imaging_app -i {test_input_path} -o output

In [ ]:
from skimage import io
output_image = io.imread("output/final_output.png")
io.imshow(output_image)

## 5. Packaging APP

- Execute the commands below in your host env </br>
****************************************************************

In [ ]:
# !sudo /raid/home/warren/miniconda3/envs/monai/bin/monai-deploy package simple_imaging_app --tag simple_monai_app:latest

In [ ]:
## Check the created image
# !sudo docker images | grep monai_app

****************************************************************

## 6. Executing Packaged APP

In [ ]:
# remove the output image first
!rm output/final_output.png

- Execute the commands below in your host env </br>
****************************************************************

In [ ]:
# !sudo /raid/home/warren/miniconda3/envs/monai/bin/monai-deploy run simple_monai_app:latest input output

****************************************************************

In [ ]:
output_image = io.imread("output/final_output.png")
io.imshow(output_image)

# Part 2: Segmentation Pipeline APP

## 1. Download and Extract Sample Data

In [ ]:
# Download ai_spleen_seg_data test data zip file
!gdown https://drive.google.com/uc?id=1GC_N8YQk_mOWN02oOzAU_2YDmNRWk--n

# After downloading ai_spleen_seg_data zip file from the web browser or using gdown,
!unzip -o "ai_spleen_seg_data_updated_1203.zip"

## 2. Creating Segmentation Operator

In [ ]:
# Create an application folder
!mkdir -p spleen_seg_app

### spleen_seg_operator.py

In [ ]:
%%writefile spleen_seg_app/spleen_seg_operator.py
import logging
from os import path

from numpy import uint8
import monai.deploy.core as md
from monai.deploy.core import (
    Application,
    DataPath,
    ExecutionContext,
    Image,
    InputContext,
    IOType,
    Operator,
    OutputContext,
)
from monai.deploy.operators.monai_seg_inference_operator import InMemImageReader, MonaiSegInferenceOperator
import monai
from monai.transforms import (
    Activationsd,
    AsDiscreted,
    Compose,
    CropForegroundd,
    EnsureChannelFirstd,
    Invertd,
    LoadImaged,
    SaveImaged,
    ScaleIntensityRanged,
    Spacingd,
    ToTensord,
)

@md.input("image", Image, IOType.IN_MEMORY)
@md.output("seg_image", Image, IOType.IN_MEMORY)
@md.env(pip_packages=["monai==0.7.0", "torch>=1.5", "numpy>=1.20", "nibabel", "typeguard", "monai-deploy-app-sdk == 0.2.0"])
class SpleenSegOperator(Operator):
    """Performs Spleen segmentation with a 3D image converted from a DICOM CT series.
    """

    def __init__(self):

        self.logger = logging.getLogger("{}.{}".format(__name__, type(self).__name__))
        super().__init__()
        self._input_dataset_key = "image"
        self._pred_dataset_key = "pred"

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):

        input_image = op_input.get("image")
        if not input_image:
            raise ValueError("Input image is not found.")

        output_path = context.output.get().path

        # This operator gets an in-memory Image object, so a specialized ImageReader is needed.
        _reader = InMemImageReader(input_image)
        pre_transforms = self.pre_process(_reader)
        post_transforms = self.post_process(pre_transforms, path.join(output_path, "prediction_output"))

        # Delegates inference and saving output to the built-in operator.
        infer_operator = MonaiSegInferenceOperator(
            (
                160,
                160,
                160,
            ),
            pre_transforms,
            post_transforms,
        )

        # Setting the keys used in the dictironary based transforms may change.
        infer_operator.input_dataset_key = self._input_dataset_key
        infer_operator.pred_dataset_key = self._pred_dataset_key

        # Now let the built-in operator handles the work with the I/O spec and execution context.
        infer_operator.compute(op_input, op_output, context)

    def pre_process(self, img_reader) -> Compose:
        """Composes transforms for preprocessing input before predicting on a model."""

        my_key = self._input_dataset_key
        return Compose(
            [
                LoadImaged(keys=my_key, reader=img_reader),
                EnsureChannelFirstd(keys=my_key),
                Spacingd(keys=my_key, pixdim=[1.0, 1.0, 1.0], mode=["blinear"], align_corners=True),
                ScaleIntensityRanged(keys=my_key, a_min=-57, a_max=164, b_min=0.0, b_max=1.0, clip=True),
                CropForegroundd(keys=my_key, source_key=my_key),
                ToTensord(keys=my_key),
            ]
        )

    def post_process(self, pre_transforms: Compose, out_dir: str = "./prediction_output") -> Compose:
        """Composes transforms for postprocessing the prediction results."""

        pred_key = self._pred_dataset_key
        return Compose(
            [
                Activationsd(keys=pred_key, softmax=True),
                AsDiscreted(keys=pred_key, argmax=True),
                Invertd(
                    keys=pred_key, transform=pre_transforms, orig_keys=self._input_dataset_key, nearest_interp=True
                ),
                SaveImaged(keys=pred_key, output_dir=out_dir, output_postfix="seg", output_dtype=uint8, resample=False),
            ]
        )

## 3. Application Class

In [ ]:
%%writefile spleen_seg_app/app.py
import logging

from spleen_seg_operator import SpleenSegOperator

import monai.deploy.core as md
from monai.deploy.core import Application, resource
from monai.deploy.operators.dicom_data_loader_operator import DICOMDataLoaderOperator
from monai.deploy.operators.dicom_seg_writer_operator import DICOMSegmentationWriterOperator
from monai.deploy.operators.dicom_series_selector_operator import DICOMSeriesSelectorOperator
from monai.deploy.operators.dicom_series_to_volume_operator import DICOMSeriesToVolumeOperator

@resource(cpu=1, gpu=1, memory="7Gi")
@md.env(pip_packages=["monai-deploy-app-sdk == 0.2.0"])
class AISpleenSegApp(Application):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def compose(self):

        study_loader_op = DICOMDataLoaderOperator()
        series_selector_op = DICOMSeriesSelectorOperator(Sample_Rules_Text)
        series_to_vol_op = DICOMSeriesToVolumeOperator()
        # Creates DICOM Seg writer with segment label name in a string list
        dicom_seg_writer = DICOMSegmentationWriterOperator(seg_labels=["Spleen"])
        # Creates the model specific segmentation operator
        spleen_seg_op = SpleenSegOperator()

        # Creates the DAG by link the operators
        self.add_flow(study_loader_op, series_selector_op, {"dicom_study_list": "dicom_study_list"})
        self.add_flow(series_selector_op, series_to_vol_op, {"study_selected_series_list": "study_selected_series_list"})
        self.add_flow(series_to_vol_op, spleen_seg_op, {"image": "image"})
        self.add_flow(series_selector_op, dicom_seg_writer, {"study_selected_series_list": "study_selected_series_list"})
        self.add_flow(spleen_seg_op, dicom_seg_writer, {"seg_image": "seg_image"})

# This is a sample series selection rule in JSON, simply selecting CT series.
# If the study has more than 1 CT series, then all of them will be selected.
# Please see more detail in DICOMSeriesSelectorOperator.
Sample_Rules_Text = """
{
    "selections": [
        {
            "name": "CT Series",
            "conditions": {
                "StudyDescription": "(.*?)",
                "Modality": "(?i)CT",
                "SeriesDescription": "(.*?)"
            }
        }
    ]
}
"""

if __name__ == "__main__":
    # Creates the app and test it standalone. When running is this mode, please note the following:
    #     -i <DICOM folder>, for input DICOM CT series folder
    #     -o <output folder>, for the output folder, default $PWD/output
    #     -m <model file>, for model file path
    # e.g.
    #     python3 app.py -i input -m model.ts
    #
    AISpleenSegApp(do_run=True)

In [ ]:
%%writefile spleen_seg_app/__main__.py
from app import AISpleenSegApp

if __name__ == "__main__":
    AISpleenSegApp(do_run=True)

## 4. APP Testing

In [ ]:
!monai-deploy exec spleen_seg_app -i dcm -o output -m model.ts

In [ ]:
import pydicom
import matplotlib.pyplot as plt

result = pydicom.dcmread('./output/dicom_seg-DICOMSEG.dcm')
plt.imshow(result.pixel_array[100], cmap='gray')

## 5. Packaging APP

- Execute the commands below in your host env </br>
****************************************************************

In [ ]:
# !sudo /raid/home/warren/miniconda3/envs/monai/bin/monai-deploy package -b nvcr.io/nvidia/pytorch:21.07-py3 spleen_seg_app --tag spleen_seg_app:latest -m model.ts

In [ ]:
# !sudo docker images | grep spleen

****************************************************************

## 6. Executing Packaged APP

- Execute the commands below in your host env </br>
****************************************************************

In [ ]:
# remove the previous output image first
!rm ./output/dicom_seg-DICOMSEG.dcm

In [ ]:
# !sudo /raid/home/warren/miniconda3/envs/monai/bin/monai-deploy run spleen_seg_app:latest dcm output

In [ ]:
result = pydicom.dcmread('./output/dicom_seg-DICOMSEG.dcm')
plt.imshow(result.pixel_array[100], cmap='gray')

****************************************************************

# Exercise
Create your own endoscopy segmentation APP!
- *Export the model as TorchScript format (model.ts)